Ruggiero Seccia

La Sapienza University of Rome

Email: ruggiero.seccia@uniroma1.it

Phone: +39 3318606535

# Optimal management of nurses' shifts (V3)


This notebook implements the final model described in the file PDF file, where we allow the nurses to work for half extra shift each day.

## importing the packages

In [1]:
import numpy as np
try:
    from docplex.mp.model import Model
except:
    !pip install docplex
from docplex.mp.model import Model

import pandas as pd

try:
    import ipywidgets
except:
    !pip install ipywidgets
from ipywidgets import interact
import ipywidgets as widgets
import copy

## Parameters specification
Let us consider a department in a hospital with a given number of nurses $N$. We want to organize their shifts for the next $T$ days, e.g. $T=7$ one week or $T=30$ next month,  and for all the followings so to minimize the effort required by the staff to satisfy the demand. By contract, each nurse $i$ has to work $H_i$ hours over the time horizon $T$ (e.g. each nurse must work at least 36 hours per week, $H=36$ and $T=7$). If the $i$th nurse works for a number of hours higher than $H_i$, then it is counted as extra work and then paid more by the healthcare structure. Each day three shifts need to be covered by the nurses: 
- Morning 
- Afternoon 
- Night

Each shift $s$ requires $R_s$ nurses and lasts $h_s$ hours. Each nurse cannot cover more than one shift per day. Moreover, we have the further constraint that if a nurse covers a night shift then they need to rest and cannot work the following day. 

Let us  consider the parameter $p_i$ which brings information about the previous period. Namely, $p_i$ is a boolean parameter such that 
\begin{equation*}
    p_i=
    \begin{cases}
    1  \qquad \text{if the } i \text{th nurse worked on the last day of the previous period} \\
    0 \qquad \text{otherwise.}
    \end{cases}
\end{equation*}

Each nurse can work more than one shift per day but up to $H^{\max}$ hours per period. In particular, each nurse is allowed to work an extra half shift before or after entering the assigned shift.


In [2]:
# number of nurses
N = 10
nurses = ['Nurse_' +str(n) for n in range(N)]
# periods to schedule
T = 7
days = ['Day_' +str(t) for t in range(T)]
# shifts
S = ['Morning', 'Afternoon', 'Night']


# standard number of hours by contract per nurse (e.g. 6 hours per day)
H_base = 6*T
H = {n:j for n in nurses for j in [H_base]*len(nurses)}

# maximum number of hours a nurse can work including the extra shifts (e.g. 10 hours per day)
H_max = 10*T
H_MAX = {n:j for n in nurses for j in [H_max]*len(nurses)}

# update some nurses values
# H['Nurse_1'] =20

# number of nurses required per shift
R = {'Morning' : 4,
     'Afternoon' : 3,
     'Night' : 3}
# duration of each shift
h = {'Morning' : 7,
     'Afternoon' : 8,
     'Night' : 9}

# list of nurses that on the last day of the previous period covered  the night shift
p_list= ['Nurse_0']
# dictionary with the values of p per each nurse
p = {n:0 for n in nurses}
# update the dictionary with p_list
for pp in p_list:
    p[pp]=1


x_i_0_1 is the boolean array with the information about the last shift of the previous week
\begin{equation*}
    x_{i01}=
    \begin{cases}
    1  \qquad \text{if the } i \text{th nurse worked on the last shift of the previous period} \\
    0 \qquad \text{otherwise.}
    \end{cases}
\end{equation*}


In [3]:
x_i_0_1 = [0]*N

x_i_4_1 is the boolean array with the information about the first shift of the next period. It should be 0 (i.e. we don't know who will work on the first shift of the next period)
\begin{equation*}
    x_{i41}=
    \begin{cases}
    1  \qquad \text{if the } i \text{th nurse works on the first shift of the first day of the next period} \\
    0 \qquad \text{otherwise.}
    \end{cases}
\end{equation*}





In [4]:
x_i_4_T = [0]*N

## Optimization model 

To formulate this optimization problem, let us introduce the binary variable $x_{ist}\in\{0,1\}$ such that 
\begin{equation*}
    x_{ist}=
    \begin{cases}
    1  \qquad \text{if nurse } i \text{th covers shift } s \text{th on day } t\text{th} \\
    0 \qquad \text{otherwise}
    \end{cases}
\end{equation*}

 We introduce the two additional integer variables $z_{ist},q_{ist}\in\{0,1\}$ such that:
\begin{equation*}
    z_{ist}=
    \begin{cases}
    1  \qquad \text{if nurse } i \text{th works the first half of the  shift } s  \text{th on day } t\text{th} \\
    0 \qquad \text{otherwise.}
    \end{cases}
\end{equation*}
\begin{equation*}
    q_{ist}=
    \begin{cases}
    1  \qquad \text{if nurse } i \text{th works the second half of the  shift } s  \text{th on day } t\text{th} \\
    0 \qquad \text{otherwise.}
    \end{cases}
\end{equation*}

Finally, we define the variable $\alpha_{st} \in R $ which represents the number of nurses that are missing to satisfy the minimum demand of the $s$th shift on the $t$th department.


In [5]:
mdl = Model('Scheduling')

# create the variables
idx_x = [(i,s,t) for i in nurses for s in S for t in days]
x = mdl.binary_var_dict(idx_x)
z = mdl.binary_var_dict(idx_x)
q = mdl.binary_var_dict(idx_x)

idx_alpha = [(s,t) for s in S for t in days]
alpha = mdl.continuous_var_dict(idx_alpha)

### Objective function
We want to find the optimal schedule $x^\star,z^\star,q^\star$ such that the proper hours worked by each nurse plus the extra shift working hours is minimized while all the department's constraints are satisfied. Of course, we are also interested in minimizing the number of shifts that cannot be covered by the nurses ($\sum_{st}\alpha_st$)

$$
\sum_{i=1}^N\sum_{s=1}^3\sum_{t=1}^T  \left(h_s x_{ist} + \frac 1 2 h_s z_{ist} +\frac 1 2 h_s q_{ist} \right) +\rho \sum_{s=1}^3\sum_{t=1}^T \alpha_{st}
$$
  
with $\rho>\max_s h_s$. Note that, even if $\alpha$ represents a discrete quantity, it is modeled as a continuous variable since at the optimum it will achieve only integer values (thanks to some further constraints defined in the following).

In [6]:
mdl.minimize(mdl.sum(x[i,s,t]*h[s] +0.5*h[s]*z[i,s,t]+0.5*h[s]*q[i,s,t] for i in nurses for s in S for t in days)
             +(max(h.values())+1000)*mdl.sum(alpha[s,t] for s in S for t in days)
            )

### Constraints
- Each person cannot cover more than one shift in the same day. 
$$ \sum_{s=1}^3x_{ist}\leq 1 \qquad \forall i,t$$
- The number of personnel per each shift in each day is satisfied. It is splitted in two because we need to consider both the first half and the second half of the shift
$$ \sum_{i=1}^N \left(x_{ist}+z_{ist}\right)+\alpha_{st} \geq R_{s} \qquad \forall s,t $$
$$ \sum_{i=1}^N \left(x_{ist}+q_{ist}\right)+\alpha_{st} \geq R_{s} \qquad \forall s, t $$

In [7]:
mdl.add_constraints(mdl.sum(x[i,s,t] for s in S) <= 1 for i in nurses for t in days);
mdl.add_constraints(mdl.sum(x[i,s,t] + z[i,s,t] for i in nurses)+alpha[s,t]>= R[s]  for s in S for t in days);
mdl.add_constraints(mdl.sum(x[i,s,t] + q[i,s,t] for i in nurses)+alpha[s,t]>= R[s]  for s in S for t in days);

- Each nurse can work a maximum number of hours $H^{\max}$ without burning out:
$$  \sum_{s=1}^3\sum_{t=1}^T h_s \left(x_{ist}+\frac 1 2 z_{ist} + \frac 1 2 q_{ist}\right) \le H^{\max}\qquad \forall\ i,s,t $$ 
- Each nurse works at minimum the number of hours required by contract (excluded the extra hours). 
$$ \sum_{s=1}^3\sum_{t=1}^T x_{ist}h_s\geq H_i \qquad \forall i $$ 

In [8]:
mdl.add_constraints(mdl.sum(h[s]*(x[i,s,t]+0.5*z[i,s,t]+0.5*q[i,s,t]) for s in S for t in days) <= H_MAX[i] for i in nurses );
mdl.add_constraints(mdl.sum(x[i,s,t]*h[s] for s in S for t in days) >= H[i] for i in nurses );

- If a nurse covers a night shift, then the next day they cannot work;
$$ x_{i3t}+\sum_{s=1}^3 x_{ist+1}\leq 1 \qquad \forall i,t $$ 
- Each nurse cannot work on the first day of the new period if they worked on the last shift of the previous period. 
$$ \sum_{s=1}^3 x_{is1}\leq (1-p_i) \qquad \forall i  $$ 

In [9]:
mdl.add_constraints( x[i,S[-1],t] + mdl.sum(x[i,s,days[j+1]] for s in S )<= 1 for i in nurses for j,t in enumerate(days[:-1]) );
mdl.add_constraints(mdl.sum(x[i,s,days[0]] for s in S ) <= (1-p[i]) for i in nurses );

-  Extra hours of the shift cannot be in consecutive days
$$ \sum_{s=1}^3 \left(z_{ist}+ z_{is(t+1)}+ q_{ist}+ q_{is(t+1)}\right)\le 1 \quad \forall i\; t=1,...,T-1$$
-  Nurses cannot work half of the shift $s$th on the day $t$th if they have already been assigned to that shift
$$ z_{ist}\leq 1-x_{ist} \qquad \forall i,s,t$$
$$ q_{ist}\le 1 - x_{ist} \qquad \forall i,s,t$$
- The number of nurses covering the first and the second hslf of each shift, have to be the same (this allow us to define the variable $\alpha_{st}$ as continuous and not as discrete)
$$ \sum_{i=1}^N z_{ist}=\sum_{i=1}^N q_{ist} \qquad \forall s,t$$


In [10]:
mdl.add_constraints(mdl.sum(z[i,s,t]+z[i,s,days[j+1]] +q[i,s,t] + q[i,s,days[j+1]] for s in S)<= 1 for i in nurses for j,t in enumerate(days[:-1]));
mdl.add_constraints(z[i,s,t]<=1-x[i,s,t] for i in nurses for s in S for t in days);
mdl.add_constraints(q[i,s,t]<=1-x[i,s,t] for i in nurses for s in S for t in days);
mdl.add_constraints(mdl.sum(z[i,s,t] for i in nurses)== mdl.sum(q[i,s,t] for i in nurses) for s in S for t in days);

-  the additional hours are joined to a proper shift
$$ z_{ist}\leq x_{is-1t} \qquad \forall i,s,t$$
$$ q_{ist}\leq x_{is+1t} \qquad \forall i,s,t$$
where $x_{i4t}=x_{i1t+1}$ and $x_{i0t} = x_{i3t-1}$. Moreover, $x_{i01}$ is known from the schedule of the previous week and $x_{i4T}$ is assumed to be zero (i.e. nobody works on the first day of the next period).

In [11]:
for idx_i,i in enumerate(nurses):
    for idx_s, s in enumerate(S):
        for idx_t,t in enumerate(days):
            if s == S[0] and t == 'Day_0':
                mdl.add_constraint(z[i,s,t]<=x_i_0_1[idx_i])
            elif s == S[0]:
                mdl.add_constraint(z[i,s,t]<=x[i,S[-1],days[idx_t-1]])
            else:
                mdl.add_constraint(z[i,s,t]<=x[i,S[idx_s-1],t])

In [12]:
for idx_i,i in enumerate(nurses):
    for idx_s, s in enumerate(S):
        for idx_t,t in enumerate(days):
            # if it is the last day of the period T
            if s == S[-1] and t == 'Day_'+str(T-1):
                mdl.add_constraint(q[i,s,t]<=x_i_4_T[idx_i])

            elif s == S[-1]:
                mdl.add_constraint(q[i,s,t]<=x[i,S[0],days[idx_t+1]])
            else:
                mdl.add_constraint(q[i,s,t]<=x[i,S[idx_s+1],t])

### Solve the problem

In [13]:
# We set a limit time for computations
mdl.set_time_limit(240)

In [14]:
mdl.print_information()
mdl.solve()
mdl.solution.solve_details

Model: Scheduling
 - number of variables: 651
   - binary=630, integer=0, continuous=21
 - number of constraints: 1123
   - linear=1123
 - parameters:
     parameters.timelimit = 240.00000000000000
 - problem type is: MILP


docplex.mp.SolveDetails(time=0.328,status='integer optimal solution')

In [15]:
status = mdl.solve_details.status == 'integer optimal solution'
status

True

## Analysing the solution
Below we provide simple tools to analyse the solution obtained

In [16]:
def sol_to_pandas(x):
    '''
    takes in input the solution of the optimization problem as a dictionary 
    returns the solution as a dataframe 
    '''
    sol = pd.DataFrame(columns = ['Nurse', 'Shift', 'Day'])
    k = 0
    for key, value in x.items():
        if value>0:
            sol.loc[k] =np.array([i for i in key])
            k+=1
    return sol

In [17]:
# transform the solution into a dataframe
x_star_dict =mdl.solution.get_value_dict(x)
sol_x = sol_to_pandas(x_star_dict)

z_star_dict =mdl.solution.get_value_dict(z)
sol_z = sol_to_pandas(z_star_dict)

q_star_dict =mdl.solution.get_value_dict(q)
sol_q = sol_to_pandas(q_star_dict)


### Is the number of nurses $N$ enough to satisfy the demand?

In [18]:
alpha_star_dict =mdl.solution.get_value_dict(alpha)
if max(list(alpha_star_dict.values()))==0:
    print('YES! The number of nurses is enough to satisfy the demand')
else:
    print('NO! The number of nurses is not enough to satisfy the demand')


YES! The number of nurses is enough to satisfy the demand


### How many hours does each nurse work?

In [19]:
worked_hours = {n:0 for n in nurses}

for i,j in sol_x.iterrows():
    worked_hours[j['Nurse']]+=h[j['Shift']]
worked_hours

{'Nurse_0': 45,
 'Nurse_1': 44,
 'Nurse_2': 46,
 'Nurse_3': 45,
 'Nurse_4': 48,
 'Nurse_5': 46,
 'Nurse_6': 47,
 'Nurse_7': 42,
 'Nurse_8': 47,
 'Nurse_9': 46}

### Maximum number of hours worked:

In [20]:
max(worked_hours.values())

48

### Average of hours worked by day

In [21]:
for i, j in worked_hours.items():
    print(i,':',j/T)

Nurse_0 : 6.428571428571429
Nurse_1 : 6.285714285714286
Nurse_2 : 6.571428571428571
Nurse_3 : 6.428571428571429
Nurse_4 : 6.857142857142857
Nurse_5 : 6.571428571428571
Nurse_6 : 6.714285714285714
Nurse_7 : 6.0
Nurse_8 : 6.714285714285714
Nurse_9 : 6.571428571428571


### How many extra hours does each nurse work?

In [22]:
extra_worked_hours = {n:0 for n in nurses}

for i,j in sol_z.iterrows():
    extra_worked_hours[j['Nurse']]+=h[j['Shift']]*0.5
for i,j in sol_q.iterrows():
    extra_worked_hours[j['Nurse']]+=h[j['Shift']]*0.5
extra_worked_hours

{'Nurse_0': 13.5,
 'Nurse_1': 9.0,
 'Nurse_2': 9.0,
 'Nurse_3': 13.0,
 'Nurse_4': 9.0,
 'Nurse_5': 4.5,
 'Nurse_6': 9.0,
 'Nurse_7': 8.0,
 'Nurse_8': 8.5,
 'Nurse_9': 13.5}

### Maximum number of extra hours worked:

In [23]:
max(extra_worked_hours.values())

13.5

### Average of extra hours worked by day

In [24]:
for i, j in extra_worked_hours.items():
    print(i,':',j/T)

Nurse_0 : 1.9285714285714286
Nurse_1 : 1.2857142857142858
Nurse_2 : 1.2857142857142858
Nurse_3 : 1.8571428571428572
Nurse_4 : 1.2857142857142858
Nurse_5 : 0.6428571428571429
Nurse_6 : 1.2857142857142858
Nurse_7 : 1.1428571428571428
Nurse_8 : 1.2142857142857142
Nurse_9 : 1.9285714285714286


### Overall hours worked

In [25]:
overall_worked_hours = worked_hours.copy()
for i,j in extra_worked_hours.items():
    overall_worked_hours[i]+=j

overall_worked_hours

{'Nurse_0': 58.5,
 'Nurse_1': 53.0,
 'Nurse_2': 55.0,
 'Nurse_3': 58.0,
 'Nurse_4': 57.0,
 'Nurse_5': 50.5,
 'Nurse_6': 56.0,
 'Nurse_7': 50.0,
 'Nurse_8': 55.5,
 'Nurse_9': 59.5}

### Visualization tool

Below we provide a tool to check the schedule. 

In [26]:
# remove warning from pandas (in the viz_tool it does what we need)
import warnings
warnings.simplefilter(action='ignore')

In [27]:
def extract_info(nurse,shift,day):
    '''
    interactive function to extract the information required:
    if a value is 'All' then it returns all the values for that specific feature
    '''
    
    global nurses,S,days
    
    
    if nurse == 'All':
        df_tmp = sol_x[(sol_x['Nurse'].isin(nurses))]
    else:
        df_tmp = sol_x[(sol_x['Nurse']==nurse)]

    if shift == 'All':
        df_tmp = df_tmp[(sol_x['Shift'].isin(S))]
    else:
        df_tmp = df_tmp[(sol_x['Shift']==shift)]

    if day == 'All':
        df_tmp = df_tmp[(sol_x['Day'].isin(days))]    
    else:
        df_tmp = df_tmp[(sol_x['Day']==day)]

    print('------------All shift------------')
    print(df_tmp)
    
    if nurse == 'All':
        df_tmp = sol_z[(sol_z['Nurse'].isin(nurses))]
    else:
        df_tmp = sol_z[(sol_z['Nurse']==nurse)]

    if shift == 'All':
        df_tmp = df_tmp[(sol_z['Shift'].isin(S))]
    else:
        df_tmp = df_tmp[(sol_z['Shift']==shift)]

    if day == 'All':
        df_tmp = df_tmp[(sol_z['Day'].isin(days))]    
    else:
        df_tmp = df_tmp[(sol_z['Day']==day)]

    print('------------First half shift------------')

    print(df_tmp)
    
    if nurse == 'All':
        df_tmp = sol_q[(sol_q['Nurse'].isin(nurses))]
    else:
        df_tmp = sol_q[(sol_q['Nurse']==nurse)]

    if shift == 'All':
        df_tmp = df_tmp[(sol_q['Shift'].isin(S))]
    else:
        df_tmp = df_tmp[(sol_q['Shift']==shift)]

    if day == 'All':
        df_tmp = df_tmp[(sol_q['Day'].isin(days))]    
    else:
        df_tmp = df_tmp[(sol_q['Day']==day)]

    print('------------Second half shift------------')
    print(df_tmp)


interact(extract_info, nurse = widgets.Dropdown(value="All",placeholder='Type something', options=nurses+['All']),
              shift=widgets.Dropdown(value='All',placeholder='Type something', options=S+['All']),
              day = widgets.Dropdown(value="All",placeholder='Type something', options=days+['All'])
        );



interactive(children=(Dropdown(description='nurse', index=10, options=('Nurse_0', 'Nurse_1', 'Nurse_2', 'Nurse…

## Minimizing the worst case scenario

 We want to modify the objective function so to minimize the worst case scenario, i.e. minimize the maximum number of hours done by a nurse. It can be easily accomplished by introducing the further continuous variable $y\in R$, rewriting the objective function as
\begin{equation*}
    \underset{x_{ist}\in\{0,1\},y\in R,\alpha_{st}\in R}{\text{min}} \quad  y + \rho \sum_{s=1}^3\sum_{t=1}^T\alpha_{st}
\end{equation*}

and introducing the set of constraints:
\begin{equation*}
    y\geq\sum_{s=1}^3\sum_{t=1}^T  \left(h_s x_{ist} + \frac 1 2 h_s z_{ist} +\frac 1 2 h_s q_{ist} \right) \qquad \forall i=1,...,N
\end{equation*}

In [28]:
# create the new variable for minimizing the worst case scenario
y = mdl.continuous_var()

# modify the objective function
mdl.minimize(y
             +(max(h.values())+1)*mdl.sum(alpha[s,t] for s in S for t in days))

# add the linear constraint
mdl.add_constraints( y>= mdl.sum(x[i,s,t]*h[s] +0.5*h[s]*z[i,s,t]+0.5*h[s]*q[i,s,t]for s in S for t in days)  for i in nurses );

mdl.solve()

docplex.mp.solution.SolveSolution(obj=55.5,values={x2:1,x3:1,x4:1,x5:1,x..

In [29]:
mdl.print_information()
mdl.solve()
mdl.solution.solve_details

Model: Scheduling
 - number of variables: 652
   - binary=630, integer=0, continuous=22
 - number of constraints: 1133
   - linear=1133
 - parameters:
     parameters.timelimit = 240.00000000000000
 - problem type is: MILP


docplex.mp.SolveDetails(time=0.078,status='integer optimal solution')

In [30]:
mdl.solve_details.status

'integer optimal solution'

## Comparing the new solution with the old one


In [31]:
# transform the solution into a dataframe
x_star_dict =mdl.solution.get_value_dict(x)
sol_x = sol_to_pandas(x_star_dict)

z_star_dict =mdl.solution.get_value_dict(z)
sol_z = sol_to_pandas(z_star_dict)

q_star_dict =mdl.solution.get_value_dict(q)
sol_q = sol_to_pandas(q_star_dict)


### Compute number of hours worked

In [32]:
# number of hours worked by each nurse 
worked_hours_wc = {n:0 for n in nurses}

for i,j in sol_x.iterrows():
    worked_hours_wc[j['Nurse']]+=h[j['Shift']]

In [33]:
# number of extra hours worked by each nurse
extra_worked_hours_wc = {n:0 for n in nurses}
# first half
for i,j in sol_z.iterrows():
    extra_worked_hours_wc[j['Nurse']]+=h[j['Shift']]*0.5
# second half
for i,j in sol_q.iterrows():
    extra_worked_hours_wc[j['Nurse']]+=h[j['Shift']]*0.5

In [34]:
# number of hours worked by each nurse overall

overall_worked_hours_wc = worked_hours_wc.copy()
for i,j in extra_worked_hours_wc.items():
    overall_worked_hours_wc[i]+=j

### Comparison with the previous solution

Hours are more equally distributed!

In [35]:
print("{0:<10s} {1:<10s} {2:<10s}".format("Nurse","Old value", "New value") )
for i in worked_hours.keys():
    print("{0:<10s} {1:<10.1f} {2:<10.1f}".format(i+":",overall_worked_hours[i],overall_worked_hours_wc[i]) )

Nurse      Old value  New value 
Nurse_0:   58.5       55.5      
Nurse_1:   53.0       55.5      
Nurse_2:   55.0       55.0      
Nurse_3:   58.0       55.5      
Nurse_4:   57.0       55.0      
Nurse_5:   50.5       55.5      
Nurse_6:   56.0       55.5      
Nurse_7:   50.0       55.5      
Nurse_8:   55.5       55.5      
Nurse_9:   59.5       54.5      


### Comparing the maximum number of hours worked overall

In [36]:
print('Maximum number of hours worked:')
print("Old value: {0:>2.2f} New value: {1:>2.2f}".format(max(overall_worked_hours.values()),max(overall_worked_hours_wc.values())) )


Maximum number of hours worked:
Old value: 59.50 New value: 55.50


### Visualization tool

Below we provide a tool to check the schedule. 

In [37]:
interact(extract_info, nurse = widgets.Dropdown(value="All",placeholder='Type something', options=nurses+['All']),
              shift=widgets.Dropdown(value='All',placeholder='Type something', options=S+['All']),
              day = widgets.Dropdown(value="All",placeholder='Type something', options=days+['All'])
        );



interactive(children=(Dropdown(description='nurse', index=10, options=('Nurse_0', 'Nurse_1', 'Nurse_2', 'Nurse…